In [1]:
!pip install google-api-python-client yt-dlp moviepy pydub google-cloud-speech nltk spacy


  Obtaining dependency information for google-api-python-client from https://files.pythonhosted.org/packages/7a/c3/2ce2d51edd0a57b7ffa0979e9c24b7f87509448c2824a08c86a439880d94/google_api_python_client-2.140.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for yt-dlp from https://files.pythonhosted.org/packages/a5/9b/f58db6c36f899c4f9c88f63aa9bc6ab676ff3d0ccbbe3201a5d3a670cfe4/yt_dlp-2024.8.6-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/170.1 kB ? eta -:--:--
     -- ------------------------------------- 10.2/170.1 kB ? eta -:--:--
     -- ------------------------------------- 10.2/170.1 kB ? eta -:--:--
     -------- ---------------------------- 41.0/170.1 kB 326.8 kB/s eta 0:00:01
     -------------------- ---------------- 92.2/170.1 kB 581.0 kB/s eta 0:00:01
     ----------------------- ------------ 112.6/170.1 kB 595.3 kB/s eta 0:00:01
     ------------------------- ---------- 122.9/170.1 kB 552.2 kB/s eta 0:00:01
     ---------------

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\ganes\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\ganes\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\ganes\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\ganes\anaconda3\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
           ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ganes\anaconda3\Lib\http\client.py", line 466, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "C:\Users\ganes\anaconda3\Lib\socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^

In [ ]:
def search_videos(keywords, max_results=5):
    request = youtube.search().list(
        q=keywords,
        part='snippet',
        type='video',
        maxResults=max_results
    )
    response = request.execute()
    videos = []
    for item in response['items']:
        video_id = item['id']['videoId']
        title = item['snippet']['title']
        description = item['snippet']['description']
        tags = item['snippet'].get('tags', [])
        videos.append((title, video_id, tags, description))
    return videos


In [ ]:
def filter_videos_by_tags(videos, required_tags):
    filtered_videos = []
    for title, video_id, tags, description in videos:
        if any(tag in required_tags for tag in tags):
            filtered_videos.append((title, video_id))
    return filtered_videos

# Example usage
required_tags = ['example_tag1', 'example_tag2']
videos = search_videos('your search query', max_results=10)
filtered_videos = filter_videos_by_tags(videos, required_tags)

for title, video_id in filtered_videos:
    download_video(video_id, f"{title}.mp4")


In [ ]:
keywords = 'tag1 tag2 tag3'
videos = search_videos(keywords, max_results=10)


In [ ]:
from moviepy.editor import VideoFileClip
from pydub import AudioSegment
from pydub.playback import play
from google.cloud import speech_v1
from google.cloud.speech_v1 import enums
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import spacy

nltk.download('vader_lexicon')
nlp = spacy.load("en_core_web_sm")

def extract_audio(video_file):
    video = VideoFileClip(video_file)
    audio = video.audio
    audio_file = video_file.replace('.mp4', '.wav')
    audio.write_audiofile(audio_file)
    return audio_file

def reduce_background_noise(audio_file):
    audio = AudioSegment.from_wav(audio_file)
    # Implement noise reduction here
    # For now, just return the same audio
    return audio_file

def speech_to_text(audio_file):
    client = speech_v1.SpeechClient()
    with open(audio_file, "rb") as audio_file:
        content = audio_file.read()
    audio = {"content": content}
    config = {"language_code": "en-US"}
    response = client.recognize(config=config, audio=audio)
    transcriptions = [result.alternatives[0].transcript for result in response.results]
    return transcriptions

def evaluate_communication(text):
    sia = SentimentIntensityAnalyzer()
    sentiment_score = sia.polarity_scores(text)
    doc = nlp(text)
    grammar_errors = [token.text for token in doc if token.pos_ == 'VERB' and token.tag_ == 'VBZ']
    return sentiment_score, grammar_errors

# Example usage
for title, video_id in videos:
    video_file = f"{title}.mp4"
    audio_file = extract_audio(video_file)
    clean_audio_file = reduce_background_noise(audio_file)
    transcriptions = speech_to_text(clean_audio_file)
    for transcription in transcriptions:
        sentiment_score, grammar_errors = evaluate_communication(transcription)
        print(f"Title: {title}")
        print(f"Sentiment Score: {sentiment_score}")
        print(f"Grammar Errors: {grammar_errors}")
